In [78]:
import os
import numpy as np
#!pip install torch
import torch
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd

In [79]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [80]:
cuda = True if torch.cuda.is_available() else False
print(cuda)

False


In [81]:
def cal_sample_weight(labels, num_class, use_sample_weight=True):
    if not use_sample_weight:
        return np.ones(len(labels)) / len(labels)
    count = np.zeros(num_class)
    for i in range(num_class):
        count[i] = np.sum(labels==i)
    sample_weight = np.zeros(labels.shape)
    for i in range(num_class):
        sample_weight[np.where(labels==i)[0]] = count[i]/np.sum(count)
    
    return sample_weight

In [82]:
def one_hot_tensor(y, num_dim):
    y_onehot = torch.zeros(y.shape[0], num_dim)
    y_onehot.scatter_(1, y.view(-1,1), 1)
    
    return y_onehot

In [83]:
def cosine_distance_torch(x1, x2=None, eps=1e-8):
    x2 = x1 if x2 is None else x2
    w1 = x1.norm(p=2, dim=1, keepdim=True)
    w2 = w1 if x2 is x1 else x2.norm(p=2, dim=1, keepdim=True)
    return 1 - torch.mm(x1, x2.t()) / (w1 * w2.t()).clamp(min=eps)

In [84]:
def to_sparse(x):
    x_typename = torch.typename(x).split('.')[-1]
    sparse_tensortype = getattr(torch.sparse, x_typename)
    indices = torch.nonzero(x)
    if len(indices.shape) == 0:  # if all elements are zeros
        return sparse_tensortype(*x.shape)
    indices = indices.t()
    values = x[tuple(indices[i] for i in range(indices.shape[0]))]
    return sparse_tensortype(indices, values, x.size())

In [85]:
#def cal_adj_mat_parameter(edge_per_node, data, metric="cosine"):
    #assert metric == "cosine", "Only cosine distance implemented"
    #dist = cosine_distance_torch(data, data)
    #parameter = torch.sort(dist.reshape(-1,)).values[edge_per_node*data.shape[0]]
    #return np.asscalar(parameter.data.cpu().numpy())
    #return parameter.data.cpu().numpy().item()
def cal_adj_mat_parameter(adj_parameter, data, metric):
    assert metric == "cosine", "Only cosine distance implemented"
    dist = cosine_distance_torch(data, data)
    edge_per_node = int(adj_parameter * data.shape[0])  # Convert to integer
    parameter = torch.sort(dist.reshape(-1,)).values[edge_per_node]
    return parameter.data.cpu().numpy().item()    

In [86]:
def graph_from_dist_tensor(dist, parameter, self_dist=True):
    if self_dist:
        assert dist.shape[0]==dist.shape[1], "Input is not pairwise dist matrix"
    g = (dist <= parameter).float()
    if self_dist:
        diag_idx = np.diag_indices(g.shape[0])
        g[diag_idx[0], diag_idx[1]] = 0
        
    return g

In [87]:
def gen_adj_mat_tensor(data, parameter, metric="cosine"):
    assert metric == "cosine", "Only cosine distance implemented"
    dist = cosine_distance_torch(data, data)
    g = graph_from_dist_tensor(dist, parameter, self_dist=True)
    adj = 1-dist if metric == "cosine" else NotImplementedError
    adj = adj * g
    adj_T = adj.transpose(0,1)
    I = torch.eye(adj.shape[0]).cuda() if cuda else torch.eye(adj.shape[0])
    adj = adj + adj_T*(adj_T > adj).float() - adj*(adj_T > adj).float()
    adj = F.normalize(adj + I, p=1)
    adj = to_sparse(adj)
    
    return adj


In [88]:
def gen_test_adj_mat_tensor(data, trte_idx, parameter, metric="cosine"):
    assert metric == "cosine", "Only cosine distance implemented"
    adj = torch.zeros((data.shape[0], data.shape[0]))
    if cuda:
        adj = adj.cuda()
    num_tr = len(trte_idx["tr"])
    
    dist_tr2te = cosine_distance_torch(data[trte_idx["tr"]], data[trte_idx["te"]])
    g_tr2te = graph_from_dist_tensor(dist_tr2te, parameter, self_dist=False)
    if metric == "cosine":
        adj[:num_tr,num_tr:] = 1-dist_tr2te
    else:
        raise NotImplementedError
    adj[:num_tr,num_tr:] = adj[:num_tr,num_tr:]*g_tr2te
    
    dist_te2tr = cosine_distance_torch(data[trte_idx["te"]], data[trte_idx["tr"]])
    g_te2tr = graph_from_dist_tensor(dist_te2tr, parameter, self_dist=False)
    if metric == "cosine":
        adj[num_tr:,:num_tr] = 1-dist_te2tr
    else:
        raise NotImplementedError
    adj[num_tr:,:num_tr] = adj[num_tr:,:num_tr]*g_te2tr # retain selected edges
    
    adj_T = adj.transpose(0,1)
    I = torch.eye(adj.shape[0])
    if cuda:
        I = I.cuda()
    adj = adj + adj_T*(adj_T > adj).float() - adj*(adj_T > adj).float()
    adj = F.normalize(adj + I, p=1)
    adj = to_sparse(adj)
    
    return adj


In [89]:
def xavier_init(m):
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
           m.bias.data.fill_(0.0)

In [90]:
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        nn.init.xavier_normal_(self.weight.data)
        if self.bias is not None:
            self.bias.data.fill_(0.0)
            
    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.sparse.mm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output    

In [91]:
class GCN_E(nn.Module):
    def __init__(self, in_dim, hgcn_dim, dropout):
        super().__init__()
        self.gc1 = GraphConvolution(in_dim, hgcn_dim[0])
        self.gc2 = GraphConvolution(hgcn_dim[0], hgcn_dim[1])
        self.gc3 = GraphConvolution(hgcn_dim[1], hgcn_dim[2])
        self.dropout = dropout

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        x = F.leaky_relu(x, 0.25)
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        x = F.leaky_relu(x, 0.25)
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc3(x, adj)
        x = F.leaky_relu(x, 0.25)
        
        return x
    

In [92]:
class Classifier_1(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.clf = nn.Sequential(nn.Linear(in_dim, out_dim))
        self.clf.apply(xavier_init)

    def forward(self, x):
        x = self.clf(x)
        return x

In [93]:
class VCDN(nn.Module):
    def __init__(self, num_view, num_cls, hvcdn_dim):
        super().__init__()
        self.num_cls = num_cls
        self.model = nn.Sequential(
            nn.Linear(pow(num_cls, num_view), hvcdn_dim),
            nn.LeakyReLU(0.25),
            nn.Linear(hvcdn_dim, num_cls)
        )
        self.model.apply(xavier_init)
        
    def forward(self, in_list):
        num_view = len(in_list)
        for i in range(num_view):
            in_list[i] = torch.sigmoid(in_list[i])
        x = torch.reshape(torch.matmul(in_list[0].unsqueeze(-1), in_list[1].unsqueeze(1)),(-1,pow(self.num_cls,2),1))
        for i in range(2,num_view):
            x = torch.reshape(torch.matmul(x, in_list[i].unsqueeze(1)),(-1,pow(self.num_cls,i+1),1))
        vcdn_feat = torch.reshape(x, (-1,pow(self.num_cls,num_view)))
        output = self.model(vcdn_feat)

        return output

In [94]:
def init_model_dict(num_view, num_class, dim_list, dim_he_list, dim_hc, gcn_dopout=0.5):
    model_dict = {}
    for i in range(num_view):
        model_dict["E{:}".format(i+1)] = GCN_E(dim_list[i], dim_he_list, gcn_dopout)
        model_dict["C{:}".format(i+1)] = Classifier_1(dim_he_list[-1], num_class)
    if num_view >= 2:
        model_dict["C"] = VCDN(num_view, num_class, dim_hc)
    return model_dict

In [95]:
def init_optim(num_view, model_dict, lr_e=1e-4, lr_c=1e-4):
    optim_dict = {}
    for i in range(num_view):
        optim_dict["C{:}".format(i+1)] = torch.optim.Adam(
                list(model_dict["E{:}".format(i+1)].parameters())+list(model_dict["C{:}".format(i+1)].parameters()), 
                lr=lr_e)
    if num_view >= 2:
        optim_dict["C"] = torch.optim.Adam(model_dict["C"].parameters(), lr=lr_c)
    return optim_dict

In [96]:

def prepare_trte_data(data_folder, view_list):
    num_view = len(view_list)
    
    labels_tr = np.loadtxt(os.path.join(data_folder, "labels_tr.csv"), delimiter=',')
    labels_te = np.loadtxt(os.path.join(data_folder, "labels_te.csv"), delimiter=',')
    labels_tr = labels_tr.astype(int)
    labels_te = labels_te.astype(int)
    data_tr_list = []
    data_te_list = []
    for i in view_list:
        data_tr_list.append(np.loadtxt(os.path.join(data_folder, str(i)+"_tr.csv"), delimiter=','))
        data_te_list.append(np.loadtxt(os.path.join(data_folder, str(i)+"_te.csv"), delimiter=','))
    num_tr = data_tr_list[0].shape[0]
    num_te = data_te_list[0].shape[0]
    data_mat_list = []
    for i in range(num_view):
        data_mat_list.append(np.concatenate((data_tr_list[i], data_te_list[i]), axis=0))
    data_tensor_list = []
    for i in range(len(data_mat_list)):
        data_tensor_list.append(torch.FloatTensor(data_mat_list[i]))
        if cuda:
            data_tensor_list[i] = data_tensor_list[i].cuda()
    idx_dict = {}
    idx_dict["tr"] = list(range(num_tr))
    idx_dict["te"] = list(range(num_tr, (num_tr+num_te)))
    data_train_list = []
    data_all_list = []
    for i in range(len(data_tensor_list)):
        data_train_list.append(data_tensor_list[i][idx_dict["tr"]].clone())
        data_all_list.append(torch.cat((data_tensor_list[i][idx_dict["tr"]].clone(),
                                       data_tensor_list[i][idx_dict["te"]].clone()),0))
    labels = np.concatenate((labels_tr, labels_te))
    
    return data_train_list, data_all_list, idx_dict, labels

In [106]:
def prepare_trte_data(data_folder, view_list):
    num_view = len(view_list)
    labels_tr = np.loadtxt(os.path.join(data_folder, "labels_tr.csv"), delimiter=',')
    labels_te = np.loadtxt(os.path.join(data_folder, "labels_te.csv"), delimiter=',')
    labels_tr = labels_tr.astype(int)
    labels_te = labels_te.astype(int)
    print("Number of views: ", num_view)
    print("Training Labels:", len(labels_tr))
    print("Testing Labels:", len(labels_te))
    
    data_tr_list = []
    data_te_list = []
    for i in view_list:
        tr_data = np.loadtxt(os.path.join(data_folder, str(i)+"_tr.csv"), delimiter=',')
        te_data = np.loadtxt(os.path.join(data_folder, str(i)+"_te.csv"), delimiter=',')
        data_tr_list.append(tr_data)
        data_te_list.append(te_data)
        print(f"View {i} Training Data Shape:", tr_data.shape)
        print(f"View {i} Testing Data Shape:", te_data.shape)
    
    
    num_tr = data_tr_list[0].shape[0]
    num_te = data_te_list[0].shape[0]
    
    data_mat_list = []
    for i in range(num_view):
        combined_data = np.concatenate((data_tr_list[i], data_te_list[i]), axis=0)
        data_mat_list.append(combined_data)
        #df = pd.DataFrame(combined_data)
        #df.to_csv(f"view_{view_list[i]}_combined.csv", index=False, header=False)
        #print(f"Saved View {view_list[i]} Combined Data to view_{view_list[i]}_combined.csv")
        #df.to_csv(f"combined_data_view_{view_list[i]}.csv", index=False)
        #print(f"Saved Combined Data to combined_data_view_{view_list[i]}.csv")
        print(f"View {view_list[i]} Combined Data Shape:", combined_data.shape)
    
    
    data_tensor_list = []
    cuda = torch.cuda.is_available()
    for i in range(len(data_mat_list)):
        tensor_data = torch.FloatTensor(data_mat_list[i])
        if cuda:
            tensor_data = tensor_data.cuda()
        data_tensor_list.append(tensor_data)
        print(f"View {view_list[i]} Tensor Data Shape:", tensor_data.shape)
    # Convert tensor to CPU, then NumPy
    #tensor_cpu = tensor_data.cpu().numpy()
    # Convert to DataFrame and save as CSV
    #df = pd.DataFrame(tensor_cpu)
    #df.to_csv(f"tensor_view_{view_list[i]}.csv", index=False)
    #print(f"Saved View {view_list[i]} Tensor Data to tensor_view_{view_list[i]}.csv")

    idx_dict = {}
    idx_dict["tr"]= list(range(num_tr)),
    idx_dict["te"]= list(range(num_tr, (num_tr + num_te)))
    print("Index Dictionary:", idx_dict["tr"])
    print("Index Dictionary:", idx_dict["te"])
    
    #data_train_list = [data_tensor_list[i][idx_dict["tr"]].clone() for i in range(len(data_tensor_list))]
    #data_all_list = [torch.cat((data_tensor_list[i][idx_dict["tr"]].clone(), data_tensor_list[i][idx_dict["te"]].clone()), 0) for i in range(len(data_tensor_list))]
    
    data_train_list = []
    data_all_list = []
    for i in range(len(data_tensor_list)):
        data_train_list.append(data_tensor_list[i][idx_dict["tr"]].clone())
        data_all_list.append(torch.cat((data_tensor_list[i][idx_dict["tr"]].clone(),
                                       data_tensor_list[i][idx_dict["te"]].clone()),0))
    for i in range(len(data_tensor_list)):
        #print(f"\n=== View {i} ===")
        #print("\nTrain Data:")
        #print(data_train_list[i])  # Prints the training tensor for the current view
        #print("\nAll Data (Train + Test):")
        #print(data_all_list[i])  # Prints the full dataset (train + test) tensor for the current view
        #print("=" * 40)  # Separator for better readability
        train_data_np = data_train_list[i].cpu().numpy()
        all_data_np = data_all_list[i].cpu().numpy()
        # Create DataFrames
        train_df = pd.DataFrame(train_data_np)
        all_df = pd.DataFrame(all_data_np)
        # Save DataFrames to CSV
        train_df.to_csv(f"view_{i}_train_data.csv", index=False, header=False)
        all_df.to_csv(f"view_{i}_all_data.csv", index=False, header=False)
        #print(f"Saved View {i} Train Data to view_{i}_train_data.csv")
        #print(f"Saved View {i} All Data to view_{i}_all_data.csv")
    labels = np.concatenate((labels_tr, labels_te))
    #print("Final Labels:", labels)
    return data_train_list, data_all_list, idx_dict, labels
def gen_trte_adj_mat(data_tr_list, data_trte_list, trte_idx, adj_parameter):
    adj_metric = "cosine"  # cosine distance
    adj_train_list = []
    adj_test_list = []
    
    for i in range(len(data_tr_list)):
        print(f"Processing View {i+1}...")
        
        adj_parameter_adaptive = cal_adj_mat_parameter(adj_parameter, data_tr_list[i], adj_metric)
        print(f"Adaptive Adjacency Parameter for View {i+1}: {adj_parameter_adaptive}")
        
        adj_train = gen_adj_mat_tensor(data_tr_list[i], adj_parameter_adaptive, adj_metric)
        adj_train_list.append(adj_train)
        print(f"Adjacency Matrix (Train) for View {i+1}:\n", adj_train)
        
        adj_test = gen_test_adj_mat_tensor(data_trte_list[i], trte_idx, adj_parameter_adaptive, adj_metric)
        adj_test_list.append(adj_test)
        print(f"Adjacency Matrix (Test) for View {i+1}:\n", adj_test)
        
    return adj_train_list, adj_test_list

# Example usage (assuming required functions and data are defined):
adj_train_list, adj_test_list = gen_trte_adj_mat(data_tr_list, data_trte_list, trte_idx, adj_parameter)
    
    
    

# Example usage:
data_folder = "dataset_rosmap"  # Change this to the actual path
view_list = [1, 2, 3]  # Modify based on available views
data_train_list, data_all_list, idx_dict, labels = prepare_trte_data(data_folder, view_list)

# Print final outputs
print("Data Train List Length:", len(data_train_list))
print("Data All List Length:", len(data_all_list))
print("Index Dictionary:", idx_dict)
print("Labels Shape:", labels.shape)


NameError: name 'data_tr_list' is not defined

In [98]:
"""
import os
import numpy as np
import pandas as pd
import torch

def prepare_trte_data(data_folder, view_list):
    num_view = len(view_list)
    labels_tr = np.loadtxt(os.path.join(data_folder, "labels_tr.csv"), delimiter=',')
    labels_te = np.loadtxt(os.path.join(data_folder, "labels_te.csv"), delimiter=',')
    labels_tr = labels_tr.astype(int)
    labels_te = labels_te.astype(int)
    
    data_tr_list = []
    data_te_list = []
    for i in view_list:
        tr_data = np.loadtxt(os.path.join(data_folder, f"{i}_tr.csv"), delimiter=',')
        te_data = np.loadtxt(os.path.join(data_folder, f"{i}_te.csv"), delimiter=',')
        data_tr_list.append(tr_data)
        data_te_list.append(te_data)
        print(f"View {i} Training Data Shape:", tr_data.shape)
        print(f"View {i} Testing Data Shape:", te_data.shape)
    
    num_tr = data_tr_list[0].shape[0]
    num_te = data_te_list[0].shape[0]
    
    data_mat_list = []
    for i in range(num_view):
        combined_data = np.concatenate((data_tr_list[i], data_te_list[i]), axis=0)
        data_mat_list.append(combined_data)
    
    data_tensor_list = []
    cuda = torch.cuda.is_available()
    for i in range(len(data_mat_list)):
        tensor_data = torch.FloatTensor(data_mat_list[i])
        if cuda:
            tensor_data = tensor_data.cuda()
        data_tensor_list.append(tensor_data)
        print(f"View {view_list[i]} Tensor Data Shape:", tensor_data.shape)
        
        # Convert tensor to CPU, then NumPy and save as CSV
        tensor_cpu = tensor_data.cpu().numpy()
        df = pd.DataFrame(tensor_cpu)
        df.to_csv(f"tensor_view_{view_list[i]}.csv", index=False)
        print(f"Saved View {view_list[i]} Tensor Data to tensor_view_{view_list[i]}.csv")
    
    idx_dict = {"tr": list(range(num_tr)), "te": list(range(num_tr, num_tr + num_te))}
    
    data_train_list = []
    data_all_list = []
    for i in range(len(data_tensor_list)):
        data_train_list.append(data_tensor_list[i][idx_dict["tr"]].clone())
        data_all_list.append(torch.cat((data_tensor_list[i][idx_dict["tr"]].clone(),
                                       data_tensor_list[i][idx_dict["te"]].clone()), 0))
        
        train_data_np = data_train_list[i].cpu().numpy()
        all_data_np = data_all_list[i].cpu().numpy()
        
        # Save train and all data as CSV
        train_df = pd.DataFrame(train_data_np)
        all_df = pd.DataFrame(all_data_np)
        train_df.to_csv(f"view_{view_list[i]}_train_data.csv", index=False, header=False)
        all_df.to_csv(f"view_{view_list[i]}_all_data.csv", index=False, header=False)
        print(f"Saved View {view_list[i]} Train Data to view_{view_list[i]}_train_data.csv")
        print(f"Saved View {view_list[i]} All Data to view_{view_list[i]}_all_data.csv")
    
    labels = np.concatenate((labels_tr, labels_te))
    return data_train_list, data_all_list, idx_dict, labels

# Example usage:
data_folder = "dataset_rosmap"  # Change this to the actual path
view_list = [1, 2, 3]  # Modify based on available views
data_train_list, data_all_list, idx_dict, labels = prepare_trte_data(data_folder, view_list)
"""


'\nimport os\nimport numpy as np\nimport pandas as pd\nimport torch\n\ndef prepare_trte_data(data_folder, view_list):\n    num_view = len(view_list)\n    labels_tr = np.loadtxt(os.path.join(data_folder, "labels_tr.csv"), delimiter=\',\')\n    labels_te = np.loadtxt(os.path.join(data_folder, "labels_te.csv"), delimiter=\',\')\n    labels_tr = labels_tr.astype(int)\n    labels_te = labels_te.astype(int)\n    \n    data_tr_list = []\n    data_te_list = []\n    for i in view_list:\n        tr_data = np.loadtxt(os.path.join(data_folder, f"{i}_tr.csv"), delimiter=\',\')\n        te_data = np.loadtxt(os.path.join(data_folder, f"{i}_te.csv"), delimiter=\',\')\n        data_tr_list.append(tr_data)\n        data_te_list.append(te_data)\n        print(f"View {i} Training Data Shape:", tr_data.shape)\n        print(f"View {i} Testing Data Shape:", te_data.shape)\n    \n    num_tr = data_tr_list[0].shape[0]\n    num_te = data_te_list[0].shape[0]\n    \n    data_mat_list = []\n    for i in range(num

In [99]:
def gen_trte_adj_mat(data_tr_list, data_trte_list, trte_idx, adj_parameter):
    adj_metric = "cosine" # cosine distance
    adj_train_list = []
    adj_test_list = []
    for i in range(len(data_tr_list)):
        adj_parameter_adaptive = cal_adj_mat_parameter(adj_parameter, data_tr_list[i], adj_metric)
        adj_train_list.append(gen_adj_mat_tensor(data_tr_list[i], adj_parameter_adaptive, adj_metric))
        adj_test_list.append(gen_test_adj_mat_tensor(data_trte_list[i], trte_idx, adj_parameter_adaptive, adj_metric))
    
    return adj_train_list, adj_test_list

In [100]:
def train_epoch(data_list, adj_list, label, one_hot_tensor, sample_weight, model_dict, optim_dict, train_VCDN=True):
    loss_dict = {}
    criterion = torch.nn.CrossEntropyLoss(reduction='none')
    for m in model_dict:
        model_dict[m].train()    
    num_view = len(data_list)
    for i in range(num_view):
        optim_dict["C{:}".format(i+1)].zero_grad()
        ci_loss = 0
        ci = model_dict["C{:}".format(i+1)](model_dict["E{:}".format(i+1)](data_list[i],adj_list[i]))
        ci_loss = torch.mean(torch.mul(criterion(ci, label),sample_weight))
        ci_loss.backward()
        optim_dict["C{:}".format(i+1)].step()
        loss_dict["C{:}".format(i+1)] = ci_loss.detach().cpu().numpy().item()
    if train_VCDN and num_view >= 2:
        optim_dict["C"].zero_grad()
        c_loss = 0
        ci_list = []
        for i in range(num_view):
            ci_list.append(model_dict["C{:}".format(i+1)](model_dict["E{:}".format(i+1)](data_list[i],adj_list[i])))
        c = model_dict["C"](ci_list)    
        c_loss = torch.mean(torch.mul(criterion(c, label),sample_weight))
        c_loss.backward()
        optim_dict["C"].step()
        loss_dict["C"] = c_loss.detach().cpu().numpy().item()
    
    return loss_dict

In [101]:
def test_epoch(data_list, adj_list, te_idx, model_dict):
    for m in model_dict:
        model_dict[m].eval()
    num_view = len(data_list)
    ci_list = []
    for i in range(num_view):
        ci_list.append(model_dict["C{:}".format(i+1)](model_dict["E{:}".format(i+1)](data_list[i],adj_list[i])))
    if num_view >= 2:
        c = model_dict["C"](ci_list)    
    else:
        c = ci_list[0]
    c = c[te_idx,:]
    prob = F.softmax(c, dim=1).data.cpu().numpy()
    
    return prob

In [102]:
def train_test(data_folder, view_list, num_class,
               lr_e_pretrain, lr_e, lr_c, 
               num_epoch_pretrain, num_epoch):
    test_inverval = 50
    num_view = len(view_list)
    dim_hvcdn = pow(num_class,num_view)
    if data_folder == 'dataset_rosmap':
        adj_parameter = 2
        dim_he_list = [200,200,100]
    #if data_folder == 'BRCA':
     #   adj_parameter = 10
      #  dim_he_list = [400,400,200]
    data_tr_list, data_trte_list, trte_idx, labels_trte = prepare_trte_data(data_folder, view_list)
    labels_tr_tensor = torch.LongTensor(labels_trte[trte_idx["tr"]])
    onehot_labels_tr_tensor = one_hot_tensor(labels_tr_tensor, num_class)
    sample_weight_tr = cal_sample_weight(labels_trte[trte_idx["tr"]], num_class)
    sample_weight_tr = torch.FloatTensor(sample_weight_tr)
    if cuda:
        labels_tr_tensor = labels_tr_tensor.cuda()
        onehot_labels_tr_tensor = onehot_labels_tr_tensor.cuda()
        sample_weight_tr = sample_weight_tr.cuda()
    adj_tr_list, adj_te_list = gen_trte_adj_mat(data_tr_list, data_trte_list, trte_idx, adj_parameter)
    dim_list = [x.shape[1] for x in data_tr_list]
    model_dict = init_model_dict(num_view, num_class, dim_list, dim_he_list, dim_hvcdn)
    for m in model_dict:
        if cuda:
            model_dict[m].cuda()
    
    print("\nPretrain GCNs...")
    optim_dict = init_optim(num_view, model_dict, lr_e_pretrain, lr_c)
    for epoch in range(num_epoch_pretrain):
        train_epoch(data_tr_list, adj_tr_list, labels_tr_tensor, 
                    onehot_labels_tr_tensor, sample_weight_tr, model_dict, optim_dict, train_VCDN=False)
    print("\nTraining...")
    optim_dict = init_optim(num_view, model_dict, lr_e, lr_c)
    for epoch in range(num_epoch+1):
        train_epoch(data_tr_list, adj_tr_list, labels_tr_tensor, 
                    onehot_labels_tr_tensor, sample_weight_tr, model_dict, optim_dict)
        if epoch % test_inverval == 0:
            te_prob = test_epoch(data_trte_list, adj_te_list, trte_idx["te"], model_dict)
            print("\nTest: Epoch {:d}".format(epoch))
            if num_class == 2:
                print("Test ACC: {:.3f}".format(accuracy_score(labels_trte[trte_idx["te"]], te_prob.argmax(1))))
                print("Test F1: {:.3f}".format(f1_score(labels_trte[trte_idx["te"]], te_prob.argmax(1))))
                print("Test AUC: {:.3f}".format(roc_auc_score(labels_trte[trte_idx["te"]], te_prob[:,1])))
            else:
                print("Test ACC: {:.3f}".format(accuracy_score(labels_trte[trte_idx["te"]], te_prob.argmax(1))))
                print("Test F1 weighted: {:.3f}".format(f1_score(labels_trte[trte_idx["te"]], te_prob.argmax(1), average='weighted')))
                print("Test F1 macro: {:.3f}".format(f1_score(labels_trte[trte_idx["te"]], te_prob.argmax(1), average='macro')))
            print()

In [103]:
if __name__ == "__main__":    
    data_folder = 'dataset_rosmap'
    view_list = [1,2,3]
    num_epoch_pretrain = 500
    num_epoch = 2500
    lr_e_pretrain = 1e-3
    lr_e = 5e-4
    lr_c = 1e-3
    adj_parameter = 0.1
    
    if data_folder == 'dataset_rosmap':
        num_class = 2
    
    train_test(data_folder, view_list, num_class,
               lr_e_pretrain, lr_e, lr_c, 
               num_epoch_pretrain, num_epoch)      

Number of views:  3
Training Labels: 245
Testing Labels: 106
View 1 Training Data Shape: (245, 200)
View 1 Testing Data Shape: (106, 200)
View 2 Training Data Shape: (245, 200)
View 2 Testing Data Shape: (106, 200)
View 3 Training Data Shape: (245, 200)
View 3 Testing Data Shape: (106, 200)
View 1 Combined Data Shape: (351, 200)
View 2 Combined Data Shape: (351, 200)
View 3 Combined Data Shape: (351, 200)
View 1 Tensor Data Shape: torch.Size([351, 200])
View 2 Tensor Data Shape: torch.Size([351, 200])
View 3 Tensor Data Shape: torch.Size([351, 200])
Index Dictionary: ([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106,

RuntimeError: The expanded size of the tensor (350) must match the existing size (106) at non-singleton dimension 1.  Target sizes: [1, 350].  Tensor sizes: [245, 106]